In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

# Crear un DataFrame de ejemplo
data = {'col1': [1, 2, None, 4, 5],
        'col2': ['A', 'B', 'C', None, 'A'],
        'col3': [10, 20, 30, 40, 50]}
df = pd.DataFrame(data)

# Definir las columnas que necesitan ser imputadas y codificadas
numeric_features = ['col1', 'col3']
categorical_features = ['col2']

# Crear un preprocesador para datos numéricos
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean', add_indicator=True))
])

# Crear un preprocesador para datos categóricos
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', add_indicator=True)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Crear un column transformer que aplica preprocesadores a columnas específicas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Crear el pipeline completo con el preprocesador y cualquier otro paso necesario
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Ajustar y transformar el DataFrame original
df_transformed = pipeline.fit_transform(df)

# Revertir la transformación para obtener un nuevo DataFrame
df_restored = pd.DataFrame(pipeline.named_steps['preprocessor'].transformers_[0][1].named_steps['imputer'].inverse_transform(df_transformed[:, :2]),
                           columns=numeric_features)

# Añadir las columnas categóricas al DataFrame restaurado
df_restored[categorical_features] = pipeline.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot'].inverse_transform(df_transformed[:, 2:])

# Mostrar los DataFrames
print("DataFrame original:")
print(df)
print("\nDataFrame transformado:")
print(df_transformed)
print("\nDataFrame restaurado:")
print(df_restored)


ValueError: Shape of the passed X data is not correct. Expected 4 columns, got 5.